In [ ]:
import nltk
import re
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.naive_bayes import MultinomialNB

from scipy.stats import uniform
from scipy.stats import randint

import seaborn as sns
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, classification_report
from nltk.stem import WordNetLemmatizer


from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import FunctionTransformer, StandardScaler

from sklearn.tree import export_graphviz
import graphviz

from sklearn.pipeline import Pipeline


In [ ]:
# Initialize the lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:

# Load the CSV file (replace this path with your actual file path)
file_path = '/Users/gowthamkishorevijay/Desktop/Playground/projects/my-venv/CEAS_08.csv'  # Adjust based on your local environment
df = pd.read_csv(file_path)

In [ ]:
def extract_urls(text):
    if not isinstance(text, str):
        return []
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, text)
    return urls

# Function to clean email body
def clean_text(text):
    if not isinstance(text, str):
       text = ''
    

    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'>+=+=+=+=+', '', text)  # Removing separators like '+=+=+=+=+=+'
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters and digits
    
    # 3. Lowercase the text
    text = text.lower()
    
    # 4. Remove non-alphabetic characters and numbers
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 5. Remove stopwords
    text_tokens = text.split()
    filtered_words = [word for word in text_tokens if word not in stop_words]
    
    return ' '.join(filtered_words)

# Apply URL extraction and data cleaning
df['extracted_urls'] = df['body'].apply(extract_urls)
df['body'] = df['body'].apply(clean_text)



print(df[['sender', 'subject', 'body', 'extracted_urls', 'label']].head())

In [ ]:
def create_pipeline(model):
    """Create a preprocessing and modeling pipeline."""
    preprocessor = ColumnTransformer(
        transformers=[
            ('body', Pipeline([
                ('convert_to_str', FunctionTransformer(lambda x: x.astype(str), validate=False)),
                ('tfidf', TfidfVectorizer(min_df=1, max_df=0.9))
            ]), 'body'),
            ('extracted_urls', Pipeline([
                ('convert_to_str', FunctionTransformer(lambda x: x.astype(str), validate=False)),
                ('tfidf', TfidfVectorizer())
            ]), 'extracted_urls'),
            # For 'message_length' column: calculate the length of the message body
            ('message_length', Pipeline([
                ('length', FunctionTransformer(lambda X: np.array(X.apply(len)).reshape(-1, 1))),  # Reshape to 2D
                ('scaler', StandardScaler())
            ]), 'body')
        ]
    )

    return Pipeline([
        ('preprocessor', preprocessor),
        ('model', model)
    ])